In [1]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

max_new_tokens = 128
SRC = "German"
TGT = "English"
# Load base model and LoRA weights
model = AutoModelForCausalLM.from_pretrained("haoranxu/ALMA-7B-R", torch_dtype=torch.float32, device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained("haoranxu/ALMA-7B-R", padding_side='left')

def create_prompt(tokenizer, input_text: str | list[str], max_length=128, src="German", tgt="English"):
    if isinstance(input_text, str):
        input_text = [input_text]
    prompt = f"Translate this from {src} to {tgt}:\n{src}: {{}}.\n{tgt}:"

    prompts = [
        prompt.format(text) for text in input_text
    ]
    
    input_ids = tokenizer(prompts, return_tensors="pt", padding=True, max_length=max_length, truncation=True).input_ids
    return input_ids

input_ids=create_prompt(tokenizer, "Ich bin ein Berliner", max_length=max_new_tokens, src=SRC, tgt=TGT)

/Users/Matey/miniconda3/envs/mlkit/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/Matey/miniconda3/envs/mlkit/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

['Translate this from Chinese to English:\nChinese: 我爱机器翻译。\nEnglish: I love machine translation.']


In [3]:
# Read input file
input_file = "Data/it-mono/train.en"
with open(input_file, "r") as f:
    # Honestly if you're using a big file you should probably read and process it line by line, instead of all at once
    input_text = f.readline().splitlines()

In [ ]:
# Translation
input_ids = create_prompt(tokenizer, input_text[:10], max_length=max_new_tokens, src=SRC, tgt=TGT)
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=5, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.6, top_p=0.9)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
